In [1]:
# PR: https://github.com/pytorch/pytorch/pull/33753

def main(device):
    import torch 
    print(torch.__version__)

    N = (1 << 29) - 2
    NA = 1 << 25
    
    dtype = torch.float

    a = torch.zeros(N, device=device, dtype=dtype, requires_grad=True)
    indices = torch.randint(0, NA, (NA, ), device=device, dtype=torch.long)
    values = torch.ones_like(indices, dtype=dtype)
    
    torch.cuda.synchronize()

    def func_forward(): 
        torch.cuda.synchronize()
        a.index_put((indices, ), values, accumulate=True)
        torch.cuda.synchronize()
        
    %timeit func_forward()

    b = a.index_put((indices, ), values, accumulate=True).sum()
    torch.cuda.synchronize()
    
    def func_backward():
        torch.cuda.synchronize()
        b.backward(retain_graph=True)    
        torch.cuda.synchronize()
        
    %timeit func_backward()

## Before

In [4]:
main("cuda:0")

1.5.0a0+4460c8b
60.7 ms ± 14.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
7.59 ms ± 15 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## After

In [2]:
main("cuda:0")

1.5.0a0+2f8ed14
65 ms ± 9.58 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
7.56 ms ± 39.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
